<a id=“title_ID”></a>
# JWST Pipeline Validation Testing Notebook: Spec2, srctype, MIRI
Instruments Affected: MIRI

## Introduction and summary of test being run

This notebook processes an image through calwebb_spec2 and examines the output of the flat_field step. The steps are as follow:

1) Set up data path and directory and image file names.

2) Set the appropriate header keywords to test.

2) Run data through the source_type step in calwebb_spec2.

3) Compare input to output SRCTYPE keyword

## Documentation
The pipeline documentation can be found here: https://jwst-pipeline.readthedocs.io/en/latest/

The pipeline code is available on GitHub: https://github.com/spacetelescope/jwst

##Data used

The data used in this test is a simulated MIRI image created using MIRISim. The documentation for MIRISim can be found here: https://wiki.miricle.org/bin/view/Public/MIRISim_Public?

Author: Mike Engesser

## Create Temporary Directory

In [1]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
data_dir = TemporaryDirectory()
os.chdir(data_dir.name)
print(data_dir)

<TemporaryDirectory '/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmplbp4ub21'>


In [2]:
import os
if 'CRDS_CACHE_TYPE' in os.environ:
    if os.environ['CRDS_CACHE_TYPE'] == 'local':
        os.environ['CRDS_PATH'] = os.path.join(os.environ['HOME'], 'crds', 'cache')
    elif os.path.isdir(os.environ['CRDS_CACHE_TYPE']):
        os.environ['CRDS_PATH'] = os.environ['CRDS_CACHE_TYPE']
print('CRDS cache location: {}'.format(os.environ['CRDS_PATH']))

CRDS cache location: /tmp/crds_cache


## Set Up Import Statements

In [3]:
from astropy.io import fits, ascii
import pytest
import numpy as np
import numpy.ma as ma

import jwst
from jwst.pipeline import Detector1Pipeline, Image2Pipeline
from jwst.datamodels import RampModel, ImageModel, dqflags
from jwst.pipeline import calwebb_image2
from jwst.pipeline import calwebb_spec2

from ci_watson.artifactory_helpers import get_bigdata
import crds
import os
import sys

In [4]:
# Specify CRDS locations and pmap
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
os.environ['TEST_BIGDATA']='https://bytesalad.stsci.edu/artifactory/'

## Print pipeline version number 

In [5]:
jwst.__version__ 

'1.7.2'

## Read in data from Artifactory

In [6]:
file = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFUSHORT_12LONGexp1_rate.fits')

## Define functions for validation and running pipeline

In [7]:
print('[*]', file=open('stpipe-log.cfg', "w"))
print('handler = file:pipeline.log', file=open('stpipe-log.cfg', "a"))
print('level = INFO', file=open('stpipe-log.cfg', "a"))

In [8]:
def run_spec2(im):
    """Creates and runs an instance of the JWST spec2 pipeline. """
    
    #only assign_wcs and srctype are run
    spec2 = calwebb_spec2.Spec2Pipeline()
    spec2.bkg_subtract.skip = True
    spec2.assign_wcs.skip = False
    spec2.imprint_subtract.skip = True
    spec2.msa_flagging.skip = True
    spec2.extract_2d.skip = True
    spec2.wavecorr.skip = True
    spec2.srctype.skip = False
    spec2.straylight.skip = True
    spec2.fringe.skip = True
    spec2.pathloss.skip = True
    spec2.barshadow.skip = True
    spec2.photom.skip = True
    spec2.resample_spec.skip = True
    spec2.cube_build.skip = True
    spec2.extract_1d.skip = True
    spec2.flat_field.skip = True
    spec2.save_results = True
        
    spec2.run(im)

In [9]:
def test_src_type(input_file, im1):
    """Passes data to spec2 pipeline and records the source type keyword of the output"""
    
    #run through spec2
    run_spec2(im1)
    
    #get the output file name
    cal_file = input_file.replace('rate.fits', 'cal.fits')
    
    #create an image model of the output and return the source type keyword
    im2 = ImageModel(cal_file)
    
    # Get the header source type value as well
    hdu=fits.open(cal_file)
    
    return im2.meta.target.source_type,hdu[1].header['SRCTYPE']

## Create an image model and test source type keywords

This test passes if none of the assertions return an error

In [10]:
im = ImageModel(file)

In [11]:
# Test the default case where no external information which should return EXTENDED
im.meta.target.source_type_apt = None
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = None

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

2022-09-20 13:32:07,553 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.


2022-09-20 13:32:07,555 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-09-20 13:32:07,557 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-09-20 13:32:07,558 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.


2022-09-20 13:32:07,559 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.


...

2022-09-20 13:32:07,746 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'save_bsub': False, 'fail_on_exception': True, 'save_wfss_esec': False, 'steps': {'bkg_subtract': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'save_combined_background': False, 'sigma': 3.0, 'maxiters': None, 'wfss_mmag_extract': None}, 'assign_wcs': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix':

2022-09-20 13:32:07,754 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'det_image_seq1_MIRIFUSHORT_12LONGexp1_rate.fits' reftypes = ['camera', 'collimator', 'disperser', 'distortion', 'filteroffset', 'fore', 'fpa', 'ifufore', 'ifupost', 'ifuslicer', 'msa', 'ote', 'regions', 'specwcs', 'wavelengthrange']


2022-09-20 13:32:07,816 - CRDS - INFO -  Fetching  /tmp/crds_cache/mappings/jwst/jwst_nirspec_superbias_0051.rmap   26.4 K bytes  (1 / 13 files) (0 / 124.3 K bytes)


Traceback (most recent call last):
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 347, in local_bestrefs
    return hv_best_references(context, parameters, reftypes)
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 367, in hv_best_references
    ctx = get_symbolic_mapping(context_file, cached=True)
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 670, in get_symbolic_mapping
    return get_pickled_mapping(   # reviewed
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks

CrdsDownloadError: Failed caching mapping files: Error fetching data for 'jwst_nirspec_superbias_0051.rmap' at CRDS server 'https://jwst-crds.stsci.edu' with mode 'http' : [Errno 28] No space left on device: '/tmp/crds_cache/mappings/jwst/jwst_nirspec_superbias_0051.rmap'

In [ ]:
# Test the case where APT specified 'POINT'
im.meta.target.source_type_apt = 'POINT'
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = None

im.meta.target.source_type_apt = 'POINT'
    
src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'POINT')
assert(src_type_hdr == 'POINT')

In [ ]:
# Test the case where APT specified 'EXTENDED'
im.meta.target.source_type_apt = 'EXTENDED'
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = None

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

In [ ]:
# If the background target keyword is set, it should always returns EXTENDED
# regardless of what APT said

im.meta.target.source_type_apt = 'POINT'
im.meta.observation.bkgdtarg = True

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

In [ ]:
# If no information from APT, but dither keywords suggest a point source it should
# return POINT

im.meta.target.source_type_apt = None
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = 'POINT-SOURCE'

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'POINT')
assert(src_type_hdr == 'POINT')

In [ ]:
# If no information from APT, but dither keywords suggest an extended source it should
# return EXTENDED

im.meta.target.source_type_apt = None
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = 'EXTENDED-SOURCE'

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

In [ ]:
# If the observation is a TSO, return POINT

im.meta.observation.bkgdtarg = False
im.meta.visit.tsovisit = True
im.meta.exposure.nints = 3

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'POINT')
assert(src_type_hdr == 'POINT')

In [ ]:
print('No problems found.')